# U-Net til segmentering av EO satellittbilder

Krever: torch, torchvision, torchgeo, geopandas, cv2, numpy, matplotlib

### Klargjøring av datasett

En god måte å starte å eksperimentere med maskinlæring på først teste med noen eksisterende datasett. I denne notebooken skal vi bruke [Landcover AI](https://paperswithcode.com/dataset/landcover-ai) som inneholder annoterte EO bilder med segmentering av 4 kategorier: Skog, vei, vann og bygninger. Datasettet kan laststes ned fra deres deres nettside direkte, men finnes også som ett av flere datasett i *torchgeo* biblioteket.

In [ ]:
# importere torchgeo
import os
import tempfile

from torch.utils.data import DataLoader
import torch
from torchgeo.datasets import LandCoverAI

In [ ]:
# laste ned og opprette trenings og test sett

landcover_train = LandCoverAI("data/landcover",split="train",download=True)
landcover_test = LandCoverAI("data/landcover",split="test",download=True)


Ta en titt i mappen for å se hvordan dataen har lagt seg.

In [ ]:
# lage dataloaders
batch_size = 8
trainloader = torch.utils.data.DataLoader(landcover_train, batch_size=batch_size, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(landcover_test, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
import matplotlib.pyplot as plt

image = landcover_train[2000]

plt.imshow(image["image"].permute(1,2,0).int())
plt.matshow(image["mask"])
plt.show()

### Sette opp U-Net

Vi sskal sette opp et klassisk U-Net, bestående av en encoder og en decoder.

![](../media/Unet.png)

For å gjøre det mer oversiktelig deler vi det opp i flere små biter. Først koder vi en standard konvolusjonsblokk. Så bruker vi blokkene til å lage dekoderen. Også lager vi en dekoder og setter det sammen til det fullstendige nettverket.

In [ ]:
import torch.nn  as nn
import torch.nn.functional as F

In [ ]:
### sette opp et en blokk for dobbelt 3x3 konvolusjonslag med ReLU aktivering
class Block(nn.Module):
	def __init__(self, inChannels, outChannels):
		super().__init__()
		self.conv1 = nn.Conv2d(inChannels, outChannels, 3,padding=1)
		self.relu = nn.ReLU()
		self.conv2 = nn.Conv2d(outChannels, outChannels, 3, padding=1)
		
	def forward(self, x):
		return self.relu(self.conv2(self.relu(self.conv1(x))))

In [ ]:
class Encoder(nn.Module):
    def __init__(self,channels=[3,32,64,128]):
        super().__init__()
        # Encoder funksjoner
        self.channels = channels
        self.blocks = nn.ModuleList([Block(channels[i],channels[i+1]) for i in range(len(channels)-1)])
        self.maxpool = nn.MaxPool2d(kernel_size=2,stride=2)


    def forward(self, x):
        # forward for encoder
        block_outputs = []
        for i in range(len(self.blocks)-1):
            x = self.blocks[i](x)
            block_outputs.append(x)
            x = self.maxpool(x)

        x = self.blocks[-1](x)
        block_outputs.append(x)
        return block_outputs

In [ ]:
class Decoder(nn.Module):
    def __init__(self,channels=[128,64,32]):
        super().__init__()
        # Decoder funksjoner
        self.channels = channels
        self.blocks = nn.ModuleList([Block(channels[i], channels[i + 1]) for i in range(len(channels) - 1)])  
        self.upConvs = nn.ModuleList([nn.ConvTranspose2d(channels[i], channels[i + 1], 2, 2) for i in range(len(channels) - 1)]) 
    def forward(self,x,block_outputs):
        x = self.upConvs[0](x)
        # forward for decoder
        for i in range(1,len(self.channels)-1):
            x = self.upConvs[i](x)
            x = torch.cat([x,block_outputs[i]],dim=1)
            x = self.blocks[i](x)

        return x
            


In [ ]:
class UNet(nn.Module):
	def __init__(self, encChannels=[3, 32, 64, 128],
		decChannels=[128,64, 32],
		nbClasses=5,
		outSize=(512,512)):
		super().__init__()
		# initialize encoder
		self.encoder = Encoder(encChannels)
		self.decoder = Decoder(decChannels)
		# initialize decoder
		self.head = nn.Conv2d(decChannels[-1], nbClasses, 1)
		self.outSize = outSize

	def forward(self, x):
		encFeatures = self.encoder(x)

		decFeatures = self.decoder(encFeatures[::-1][0], encFeatures[::-1][1:])

		map = self.head(decFeatures)

		map = F.interpolate(map,(512,512))
		return map

In [ ]:
test_im = torch.zeros(4,3,512,512)

out = UNet()(test_im)


In [ ]:
#from torchinfo import summary

#model = UNet()

#summary(model,(4,3,512,512))

### Sette opp en treningsloop

Vi er nå klare for å trene nettverket vårt.

In [ ]:
net = UNet()
net.train()
if torch.cuda.is_available():
    net.to("cuda") 

In [ ]:
# definer tapsfunksjon og optimeringsalgoritme
import torch.optim as optim

lossfunc = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [ ]:
# lage en accuracy test
import numpy as np
def test_accuracy(batch_lim=None):
    acc = []
    with torch.no_grad():
        for i,batch in enumerate(testloader):
            images = batch["image"]/255
            labels = batch["mask"]
            if torch.cuda.is_available():
                out = net(images.to("cuda")).to("cpu")
            else:
                out = net(images)
            _, prediction = torch.max(torch.softmax(out,dim=1),dim=1)
            pixel_accuracy = torch.sum(torch.eq(prediction,labels))/prediction.numel()
            acc.append(pixel_accuracy)
            if batch_lim is None:
                pass
            elif i > batch_lim:
                break
    return np.mean(np.array(acc))

In [ ]:
loss_log = []
accuracy_log = []
current_best = 0.2
for e in range(5):
    # lage variabler for totalt tap
    running_loss = 0
    for i,batch in enumerate(trainloader):
        optimizer.zero_grad()

        images = batch["image"]/255
        labels = batch["mask"]

        if torch.cuda.is_available():
            images = images.to("cuda")
            labels = labels.to("cuda")

        out = net(images)
        loss = lossfunc(out,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 200 == 199 or i == 0:
            if i == 0:
                avg_loss = running_loss
            else:
                avg_loss = running_loss/(200)
            print(f"ep: {e+1}: batch: {(i+1)} Avg batch loss {avg_loss}")
            loss_log.append(avg_loss)
            running_loss = 0
        
            accuracy = test_accuracy()
            print(f"Accuracy {accuracy}")
            accuracy_log.append(accuracy)
            if accuracy > current_best:
                torch.save(net.state_dict(),"models/best_unet_X.pt")
                current_best = accuracy
                print("new model saved")

Vis frem loss - loggen

In [ ]:
plt.plot(loss_log)
plt.show()
plt.plot(accuracy_log)
plt.show()

Og inspiser noen resultater

In [ ]:
import torchvision
import numpy as np
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def matshow(img):
    npimg = img.numpy()[0]
    plt.matshow(npimg,vmin=0,vmax=4)
    plt.show()


def view_batch(batch):
    image = batch["image"]/255
    labels = batch["mask"]
    if torch.cuda.is_available():
        out = net(image.to("cuda")).to("cpu")
    else:
        out = net(image)
        
    _, prediction = torch.max(torch.softmax(out,dim=1),dim=1)
    accuracy = torch.sum(torch.eq(prediction,labels))/prediction.numel()    

    image = image/255
    labels = labels.unsqueeze(1).repeat(1,3,1,1)
    prediction = prediction.unsqueeze(1).repeat(1,3,1,1)


    imshow(torchvision.utils.make_grid(image*255))
    matshow(torchvision.utils.make_grid(labels))
    matshow(torchvision.utils.make_grid(prediction))

    print(f"Accuracy: {accuracy:.2f}%")

Last inn den beste modellen

In [ ]:
# definer et nett
net = UNet()
if torch.cuda.is_available():
    net = net.to("cuda")

#print(test_accuracy(10))

# last inn beste modell
if torch.cuda.is_available():
    net.load_state_dict(torch.load("models/bestUnet.pt",map_location="cuda"))
else:
    net.load_state_dict(torch.load("models/bestUnet.pt",map_location="cpu"))

#print(test_accuracy(10))


In [ ]:
loop = iter(testloader)

In [ ]:
batch = next(loop)

view_batch(batch)

Hva om vi vil kjøre nettverket vårt på georeferert satellittdata?
Vi kan ta en av GeoTiffene fra mappen *images*, og laste den inn som et `GeoImage`.

In [ ]:
# velg nett
net = UNet()
if torch.cuda.is_available():
    net = net.to("cuda")
    net.load_state_dict(torch.load("models/bestUnet.pt",map_location="cuda"))
else:
    net.load_state_dict(torch.load("models/bestUnet.pt",map_location="cpu"))
net.eval()


In [ ]:
# legg til src i path
from pathlib import Path
import sys

src_path = Path(".").absolute().parent / Path('src')
sys.path.append(src_path.__str__())

from geoutils import GeoImage

In [ ]:
fil = "data/landcover/images/N-33-60-D-d-1-2.tif"

image = GeoImage(fil)

#finn høyde og bredde
c,H,W = image.data.shape

# lag en maske som er like stor som bildet
mask_data = np.zeros((1,H,W))

Vårt nettverk tar inn bilder av størrelse 512x512, vi må derfor gå gjennom det originale bildet og gjøre nettverket på såkalte "tiles" eller "chips" av den størrelsen. Etter hvert som vi regner ut segmenteringsmaskene, fyller vi dem inn i `mask_data`.

In [ ]:
for i in range(0,H-512,512):
    for j in range(0,W-512,512):
        local_image=image.data[:,i:i+512,j:j+512]/255
        local_image = torch.from_numpy(local_image).unsqueeze(0).float()
        if torch.cuda.is_available():
            out = net(local_image.to("cuda")).to("cpu")
        else:
            out = net(local_image)
        
        _, prediction = torch.max(torch.softmax(out,dim=1),dim=1)
        mask_data[0,i:i+512,j:j+512] = prediction



Nå kan vi skrive ut masken vi har laget til koordinetene til input-bildet

In [ ]:
from geoutils import CopyGeoTransform

mask_image = CopyGeoTransform(image,mask_data,"segmentert_maske.tif")